## Install (Colab Only)
Skip this step when running locally.

In [1]:
# install
!pip install pyepo
!pip install gurobipy
!pip install mpax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.0/131.0 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s 

Remove the problematic hook in Google Colab

In [2]:
import sys
sys.meta_path = [hook for hook in sys.meta_path if not any(keyword in str(hook) for keyword in ["google.colab"])]

# Optimization Model

PyEPO is a library for end-to-end predict-then-optimize with linear objective functions and unknown cost coefficients. The core component is the differentiable optimization solver, which computes the gradient of the predicted cost with respect to the optimal solution.

For ease of use, the library extends PyTorch's autograd modules to wrap the solver. Users can choose any solver or algorithm and obtain the optimal solution while computing the gradient through backpropagation.

This tutorial demonstrates how to build optimization models ``optModel`` with PyEPO.

## 1 Problem Example: Shortest Path

Consider a 5x5 grid network, represented as a weighted graph. The figure shows that each node has top, bottom, left, and right neighbors. We aim to find the shortest path from left top to right bottom.

<img src="https://github.com/khalil-research/PyEPO/blob/main/images/shortestpath.png?raw=1" width="500">

The weighted graph includes 25 nodes and 40 edges. The weights of the edges are the costs of the path. Let's set random weights.

In [3]:
import random
# random seed
random.seed(42)
# set random cost for test
cost = [random.random() for _ in range(40)]

## 2 Introduction to optModel

``optModel`` is a module in the PyEPO library. It is not a solver itself but serves as a container for a solver or algorithm. This design gives users flexibility to choose any solver. ``optModel`` treats solvers as black boxes and provides three interfaces: ``_getModel``, ``setObj``, and ``solve``. Other PyEPO modules use ``optModel`` for tasks such as training and testing.

Methods of ``optModel``:
- ``_getModel``: Build and return the optimization solver and its decision variables.
- ``setObj``: Set the objective function given a cost vector.
- ``solve``: Solve the optimization problem and return the optimal solution and objective value.

### 2.1 Build Model with NetworkX and Dijkstra Algorithm

We can build the graph with NetworkX and use Dijkstra's method to compute the shortest weighted path between two nodes in a graph.

In [4]:
import networkx as nx
import numpy as np
from pyepo.model.opt import optModel

class myShortestPathModel1(optModel):

    def __init__(self, grid):
        """
        Args:
            grid (tuple): size of grid network
        """
        self.grid = grid
        self.arcs = self._getArcs()
        super().__init__()

    def _getModel(self):
        """
        A method to build model

        Returns:
            tuple: optimization model and variables
        """
        # build graph as optimization model
        g = nx.Graph()
        # add arcs as variables
        g.add_edges_from(self.arcs, cost=0)
        return g, g.edges

    def setObj(self, c):
        """
        A method to set objective function

        Args:
            c (ndarray): cost of objective function
        """
        # set weights for edges
        for i, e in enumerate(self.arcs):
            self._model.edges[e]["cost"] = c[i]

    def solve(self):
        """
        A method to solve model

        Returns:
            tuple: optimal solution (list) and objective value (float)
        """
        # dijkstra
        path = nx.shortest_path(self._model, weight="cost", source=0, target=self.grid[0]*self.grid[1]-1)
        # convert path into active edges
        edges = []
        u = 0
        for v in path[1:]:
            edges.append((u,v))
            u = v
        # init sol & obj
        sol = np.zeros(self.num_cost)
        obj = 0
        # convert active edges into solution and obj
        for i, e in enumerate(self.arcs):
            if e in edges:
                sol[i] = 1 # active edge
                obj += self._model.edges[e]["cost"] # cost of active edge
        return sol, obj

    def _getArcs(self):
        """
        A helper method to get list of arcs for grid network

        Returns:
            list: arcs
        """
        arcs = []
        for i in range(self.grid[0]):
            # edges on rows
            for j in range(self.grid[1] - 1):
                v = i * self.grid[1] + j
                arcs.append((v, v + 1))
            # edges on columns
            if i == self.grid[0] - 1:
                continue
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                arcs.append((v, v + self.grid[1]))
        return arcs

In [5]:
# solve model
optmodel = myShortestPathModel1(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Obj: 2.2869938328922332
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


### 2.2 Build Model with GurobiPy and Linear Programming

``optModel`` also allows users to use optimization modeling languages such as GurobiPy and Pyomo. With ``optGrbModel``, users can build a Gurobi LP model by overriding ``_getModel``. Similarly, ``optOmoModel`` supports Pyomo.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from pyepo.model.grb import optGrbModel

class myShortestPathModel2(optGrbModel):

    def __init__(self, grid):
        """
        Args:
            grid (tuple of int): size of grid network
        """
        self.grid = grid
        self.arcs = self._getArcs()
        super().__init__()

    def _getModel(self):
        """
        A method to build Gurobi model

        Returns:
            tuple: optimization model and variables
        """
        # create a model
        m = gp.Model("shortest path")
        # variables
        x = m.addVars(self.arcs, name="x")
        # sense
        m.modelSense = GRB.MINIMIZE
        # constraints
        for i in range(self.grid[0]):
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                expr = 0
                for e in self.arcs:
                    # flow in
                    if v == e[1]:
                        expr += x[e]
                    # flow out
                    elif v == e[0]:
                        expr -= x[e]
                # source
                if i == 0 and j == 0:
                    m.addConstr(expr == -1)
                # sink
                elif i == self.grid[0] - 1 and j == self.grid[0] - 1:
                    m.addConstr(expr == 1)
                # transition
                else:
                    m.addConstr(expr == 0)
        return m, x


    def _getArcs(self):
        """
        A helper method to get list of arcs for grid network

        Returns:
            list: arcs
        """
        arcs = []
        for i in range(self.grid[0]):
            # edges on rows
            for j in range(self.grid[1] - 1):
                v = i * self.grid[1] + j
                arcs.append((v, v + 1))
            # edges in columns
            if i == self.grid[0] - 1:
                continue
            for j in range(self.grid[1]):
                v = i * self.grid[1] + j
                arcs.append((v, v + self.grid[1]))
        return arcs

In [7]:
# solve model
optmodel = myShortestPathModel2(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Restricted license - for non-production use only - expires 2026-11-23
Obj: 2.2869939003139734
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


### 2.3  Pre-defined Gurobi Model

PyEPO includes several pre-defined optimization models built with GurobiPy and Pyomo.

In [8]:
# shortest path on the grid network
from pyepo.model.grb import shortestPathModel

In [9]:
# solve model
optmodel = shortestPathModel(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Obj: 2.2869939003139734
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


### 2.4 Pre-defined MPAX Model
PyEPO contains several pre-defined optimization models with MPAX.

In [10]:
# shortest path on the grid network
from pyepo.model.mpax import shortestPathModel

In [13]:
# solve model
optmodel = shortestPathModel(grid=(5,5)) # init model
optmodel.setObj(cost) # set objective function
sol, obj = optmodel.solve() # solve
# print res
print('Obj: {}'.format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Obj: 2.286993980407715
(0, 1)
(1, 2)
(2, 3)
(3, 8)
(8, 9)
(9, 14)
(14, 19)
(19, 24)


## 3 Introduction to optMpaxModel

In this section, we introduce how to model and solve optimization problems in **PyEPO** using [`MPAX`](https://github.com/MIT-Lu-Lab/MPAX), a PDHG (Primal-Dual Hybrid Gradient) optimization framework.  

`optMpaxModel` is a PyEPO optimization model that leverages `MPAX `for solving **LP problems** (**not ILP**) using the PDHG algorithm. Unlike traditional solvers, MPAX is highly efficient on GPU, making it an excellent choice for large-scale decision-focused learning tasks.



### 3.1 Mathematical Formulation

The model solves a linear program (LP) of the form:

$$
\min_{\mathbf{x}} \mathbf{c}^\top \mathbf{x}
$$

Subject to:  

- Equality constraints: $\mathbf{A} \mathbf{x} = \mathbf{b} $  
- Inequality constraints: $ \mathbf{G} \mathbf{x} \geq \mathbf{h} $  
- Variable bounds: $ \mathbf{l} \leq \mathbf{x} \leq \mathbf{u} $  

Where:  
- $ c $ is the cost vector (objective function coefficients)  
- $ A, b $ define the equality constraints
- $ G, h $ define the inequality constraints
- $ l, u $ define the variable bounds

### 3.2 Model Parameters

`optMpaxModel` only accepts model coefficients in matrix and vector form. This means:
- Constraints must be explicitly provided as matrices and vectors (no symbolic constraints).
- No direct support for constraints in Python functions (e.g., defining Ax = b in a callable function).
- If `A` and `b` are not provided, it means **there are no equality constraints** in the problem.
- If `G` and `h` are not provided, it means **there are no inequality constraints** in the problem.
- The default value for `l` (lower bound) is `0`, meaning all variables are **non-negative** unless specified otherwise.
- The default value for `u` (upper bound) is infinity, meaning variables are unbounded unless explicitly set.

In [14]:
import numpy as np
import torch
# random seed for reproducibility
np.random.seed(42)
# number of variables
num_vars = 10
# random equality and inequality constraints
A = np.random.rand(3, num_vars)  # 3 equality constraints
b = np.random.rand(3)
G = np.random.rand(5, num_vars)  # 5 inequality constraints
h = np.random.rand(5)

### 3.3 Create Model and Solve

Now, we create an optMpaxModel instance with the defined constraints.

In addition to model parameters, we also have two additional settings:  
- **`use_sparse_matrix`** (`bool`, default: `True`) – Stores constraint matrices in a sparse format or not. Choosing the right matrix type is crucial for optimization efficiency.  
- **`minimize`** (`bool`, default: `True`) – Defines whether to minimize the objective function.  

In [15]:
from pyepo.model.mpax import optMpaxModel
optmodel = optMpaxModel(A=A, b=b, G=G, h=h, use_sparse_matrix=False, minimize=True)

In [16]:
# generate a random cost vector
cost = np.random.rand(num_vars)
# solve the model
optmodel.setObj(cost)
sol, obj = optmodel.solve()
# print results
print(f"Objective Value: {obj}")
print(f"Solution: {sol}")

Objective Value: 0.8743340969085693
Solution: tensor([0.0262, 0.0000, 0.0000, 0.3298, 0.0000, 0.0000, 0.5673, 0.0029, 0.0000,
        0.4142])
